# Лабораторная работа 2. Классификация текстовых данных. Рекуррентные нейронные сети Many To One


## Задание
- Осознать теоретические основы работы рекуррентных нейронных сетей;

- изучить возможности и инструменты PyTorch для построения и использования рекуррентных блоков (RNN. LSTM, GRU);

- решить задачу классификации текстовых документов с использованием рекуррентных нейронных сетей вида Many To One (многие к одному).



## Ход работы
1. Загрузите датасет из ЛР1. Датасет большой, рекомендуется для работы взять его часть (далее - корпус). Если вам не нравится этот набор данных, можете использовать другой, по своему вкусу.

2. Рассмотрите каждый объект корпуса как отдельный текстовый документ. Выполните предварительную обработку каждого объекта.

3. Выполните векторизацию каждого документа любым способом (можно с помощью своих моделей Word2Vec, при условии, что Word2Vec модели обучались на той же выборке документов; можно использовать другие библиотеки и модели).

4. Закодируйте значения целевого признака (категория обращения). Разделите данные на обучающую и тестирующую выборку.

5. С помощью PyTorch соберите и обучите три нейронных сети для решения задачи классификации (предсказания категории обращения); количество слоев и значения других параметров конфигурируете по вашему усмотрению:

- сеть на базе блока RNN;

- сеть на базе блока LSTM;

- сеть на базе блока GRU.

6.  Используя известные вам метрики классификации и тестирующую выборку оцените качество каждой из обученных моделей. Сделайте вывод о том, какая из моделей на ваших данных отработала лучше других.

In [151]:
!pip install pymorphy3

In [152]:
import pandas as pd
import torch
from torch import nn
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import re
from pymorphy3 import MorphAnalyzer
import string
from gensim.models import Word2Vec
import numpy as np
from sklearn.decomposition import PCA
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [153]:
torch.cuda.is_available()

True

In [154]:
torch.__version__

'2.5.1+cu121'

### 1. Загрузите датасет из ЛР1. Датасет большой, рекомендуется для работы взять его часть (далее - корпус). Если вам не нравится этот набор данных, можете использовать другой, по своему вкусу.

In [155]:
data = pd.read_csv("Petitions.csv")

In [156]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59889 entries, 0 to 59888
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    59889 non-null  int64 
 1   public_petition_text  59889 non-null  object
 2   reason_category       59889 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [157]:
data.head(1000)

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство
...,...,...,...
995,3279475,Повреждения дорожного покрытия проезжей части,Благоустройство
996,3356222,дворник убрал песок у подъезда и высыпал его н...,Благоустройство
997,3276541,Нужно заделать ямы во дворе,Благоустройство
998,3032938,демонтировать решетку,Нарушение правил пользования общим имуществом


In [158]:
data = data.head(10000)

In [159]:
data.drop(["id"], axis=1, inplace=True)

<ipython-input-159-0544d42fdce7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(["id"], axis=1, inplace=True)


### 2. Рассмотрите каждый объект корпуса как отдельный текстовый документ. Выполните предварительную обработку каждого объекта.

Удаление всех символов, которые не являются цифрами, буквами и пробелами, и токенизация

In [160]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [161]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [162]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [163]:
morph = MorphAnalyzer()
stopwords = stopwords.words("russian")


def preProccesingData(text):
    only_letters = re.sub("[^\s^\w]+", " ", text)
    tokenized = word_tokenize(only_letters)
    normalized = [morph.normal_forms(word)[0] for word in tokenized]
    no_stop_words = [
        word
        for word in normalized
        if ((word not in string.punctuation) and (word not in stopwords) and word.isalpha())
    ]
    return no_stop_words

In [164]:
data['public_petition_text'] = data['public_petition_text'].apply(preProccesingData).tolist()
data[:10]

,public_petition_text,reason_category
0,"[снег, дорога]",Благоустройство
1,"[очистить, кабельный, киоск, реклама]",Благоустройство
2,"[просить, убрать, всё, дерево, кустарник, кото...",Благоустройство
3,"[неудовлетворительный, состояние, парадный, на...",Содержание МКД
4,[граффити],Благоустройство
5,"[необходимо, проверить, законность, установка,...",Незаконная информационная и (или) рекламная ко...
6,"[уборка, производиться, лестница, очень, грязн...",Содержание МКД
7,[мусор],Благоустройство
8,"[отсутствовать, освещение, лестничный, площадк...",Содержание МКД
9,"[делать, благоустройство, никто, убирать, мусо...",Благоустройство


### 3. Выполните векторизацию каждого документа любым способом (можно с помощью своих моделей Word2Vec, при условии, что Word2Vec модели обучались на той же выборке документов; можно использовать другие библиотеки и модели).

In [165]:
X = list(data['public_petition_text'])
y_str = list(data['reason_category'])
X

[['снег', 'дорога'],
 ['очистить', 'кабельный', 'киоск', 'реклама'],
 ['просить',
  'убрать',
  'всё',
  'дерево',
  'кустарник',
  'который',
  'выйти',
  'предел',
  'газон',
  'пешеходный',
  'зона',
  'начинать',
  'подъезд',
  'подъезд',
  'фасад',
  'дом',
  'сторона',
  'ул',
  'наличный'],
 ['неудовлетворительный', 'состояние', 'парадный', 'надпись', 'дверь', 'этаж'],
 ['граффити'],
 ['необходимо',
  'проверить',
  'законность',
  'установка',
  'вывеска',
  'фасад',
  'мкд',
  'адрес',
  'проспект',
  'непокорённый',
  'случай',
  'вывеска',
  'установить',
  'незаконно',
  'её',
  'необходимо',
  'демонтировать'],
 ['уборка',
  'производиться',
  'лестница',
  'очень',
  'грязно',
  'весь',
  'этаж',
  'вплоть',
  'го',
  'звонок',
  'жкс',
  'дать',
  'результат'],
 ['мусор'],
 ['отсутствовать', 'освещение', 'лестничный', 'площадка', 'этаж', 'парадный'],
 ['делать', 'благоустройство', 'никто', 'убирать', 'мусор', 'ежедневно'],
 ['просьба', 'закрасить'],
 ['реклама', 'забор']

In [166]:
all_words = [word for document in X for word in document]

In [167]:
all_words[:10]

['снег',
 'дорога',
 'очистить',
 'кабельный',
 'киоск',
 'реклама',
 'просить',
 'убрать',
 'всё',
 'дерево']

In [168]:
vocab = set(all_words)
vocab_len = len(vocab)
vocab_len

7771

In [169]:
id_word = dict(enumerate(vocab))
id_word

{0: 'взудитие',
 1: 'утверждение',
 2: 'ношение',
 3: 'стать',
 4: 'зайцев',
 5: 'гск',
 6: 'инстанция',
 7: 'новорождённый',
 8: 'бездейстовать',
 9: 'неудовлетворительно',
 10: 'чебышевский',
 11: 'частично',
 12: 'достаточно',
 13: 'параднойгрязь',
 14: 'судебный',
 15: 'пища',
 16: 'снаружи',
 17: 'петербург',
 18: 'таракашка',
 19: 'прохладный',
 20: 'указываться',
 21: 'тройка',
 22: 'зацементировать',
 23: 'облицевать',
 24: 'неудолетворительный',
 25: 'блюхер',
 26: 'молодец',
 27: 'заскорузлый',
 28: 'сидение',
 29: 'грязно',
 30: 'оборванный',
 31: 'здравствуйте',
 32: 'нерасчистить',
 33: 'инвалидный',
 34: 'наружу',
 35: 'вышеупомянутый',
 36: 'навес',
 37: 'конкретно',
 38: 'колпак',
 39: 'добавиться',
 40: 'президент',
 41: 'теплосеть',
 42: 'перекидывание',
 43: 'небо',
 44: 'сомнительный',
 45: 'зашпаклевать',
 46: 'верно',
 47: 'существенный',
 48: 'щелчок',
 49: 'статус',
 50: 'вышеизложенный',
 51: 'ежевика',
 52: 'круглосуточно',
 53: 'первозданный',
 54: 'отсыпка',

In [170]:
word_id = {v: k for k, v in id_word.items()}
word_id

{'взудитие': 0,
 'утверждение': 1,
 'ношение': 2,
 'стать': 3,
 'зайцев': 4,
 'гск': 5,
 'инстанция': 6,
 'новорождённый': 7,
 'бездейстовать': 8,
 'неудовлетворительно': 9,
 'чебышевский': 10,
 'частично': 11,
 'достаточно': 12,
 'параднойгрязь': 13,
 'судебный': 14,
 'пища': 15,
 'снаружи': 16,
 'петербург': 17,
 'таракашка': 18,
 'прохладный': 19,
 'указываться': 20,
 'тройка': 21,
 'зацементировать': 22,
 'облицевать': 23,
 'неудолетворительный': 24,
 'блюхер': 25,
 'молодец': 26,
 'заскорузлый': 27,
 'сидение': 28,
 'грязно': 29,
 'оборванный': 30,
 'здравствуйте': 31,
 'нерасчистить': 32,
 'инвалидный': 33,
 'наружу': 34,
 'вышеупомянутый': 35,
 'навес': 36,
 'конкретно': 37,
 'колпак': 38,
 'добавиться': 39,
 'президент': 40,
 'теплосеть': 41,
 'перекидывание': 42,
 'небо': 43,
 'сомнительный': 44,
 'зашпаклевать': 45,
 'верно': 46,
 'существенный': 47,
 'щелчок': 48,
 'статус': 49,
 'вышеизложенный': 50,
 'ежевика': 51,
 'круглосуточно': 52,
 'первозданный': 53,
 'отсыпка': 54,

In [171]:
w2v = Word2Vec(sentences=X, min_count=1, vector_size=32, epochs=50)

In [172]:
word_vectors = {word: w2v.wv[word] for word in vocab}

In [173]:
word_vectors

{'взудитие': array([-0.12857096, -0.1821478 , -0.08437478,  0.15940414,  0.1322826 ,
        -0.05456191,  0.20577869,  0.05710222, -0.27503088, -0.06156126,
         0.08349769, -0.17244886,  0.08279859, -0.03958371,  0.00427412,
         0.01308259,  0.0626841 , -0.02038973,  0.00573065,  0.26035845,
         0.19766244,  0.30924967,  0.13790756, -0.0784334 ,  0.06865068,
         0.0192849 , -0.27631053, -0.00213539, -0.0187077 , -0.21167736,
         0.00176252, -0.00755135], dtype=float32),
 'утверждение': array([ 0.03236619, -0.97494036, -2.579772  , -0.12325174,  1.8898939 ,
        -0.32748672,  0.9215149 , -0.4656426 , -1.2852348 , -0.9579957 ,
         0.9014096 , -0.99653345, -0.76505834,  1.4386164 , -0.6214017 ,
         0.34042814, -0.45976612,  0.58687603,  0.3044383 ,  2.4436803 ,
         0.11465684,  1.0793157 , -1.0678763 ,  1.1741205 ,  0.10640617,
         0.08737281, -0.99143976, -0.48404813, -0.16276431, -0.89135426,
        -0.74416935,  0.8076415 ], dtype=float

### 4. Закодируйте значения целевого признака (категория обращения). Разделите данные на обучающую и тестирующую выборку.

In [174]:
encoder = LabelEncoder()
y = encoder.fit_transform(y_str)
y

array([0, 0, 0, ..., 0, 8, 0])

In [175]:
corpus_vec = [[word_vectors[word] for word in doc] for doc in X]

In [176]:
len(corpus_vec)

10000

In [177]:
corpus_vec_one_dim = torch.nn.utils.rnn.pad_sequence([torch.tensor(x) for x in corpus_vec], batch_first=True)

In [178]:
len(corpus_vec_one_dim)

10000

In [179]:
X_train, X_test, y_train, y_test = train_test_split(corpus_vec_one_dim, np.array(y), test_size=0.1, stratify=y, shuffle=True)

In [180]:
train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32).cuda(), torch.from_numpy(y_train).type(torch.long).cuda())
train_dl = DataLoader(train_ds, batch_size=256, shuffle=True)

<ipython-input-180-ac4091729f68>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32).cuda(), torch.from_numpy(y_train).type(torch.long).cuda())


In [181]:
test_ds = TensorDataset(torch.tensor(X_test, dtype=torch.float32).cuda(), torch.tensor(y_test).reshape(-1,1).type(torch.long).cuda())
test_dl = DataLoader(test_ds, batch_size=256, shuffle=True)

<ipython-input-181-12742d7bbe37>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_ds = TensorDataset(torch.tensor(X_test, dtype=torch.float32).cuda(), torch.tensor(y_test).reshape(-1,1).type(torch.long).cuda())


### 5. С помощью PyTorch соберите и обучите три нейронных сети для решения задачи классификации (предсказания категории обращения); количество слоев и значения других параметров конфигурируете по вашему усмотрению:

#### RNN

In [182]:
X_train.shape

torch.Size([9000, 191, 32])

In [183]:
y_train

array([ 0,  0,  0, ...,  0, 11, 11])

In [184]:
INPUT_SIZE = X_train.shape[2]
HIDDEN_SIZE = 128

In [185]:
class RNNModel(nn.Module):
    def __init__(self, input_size: int, hidden_size: int) -> None:
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 15)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, h = self.rnn(x)
        y = self.linear(h.squeeze(0))
        return y

rnn = RNNModel(INPUT_SIZE, HIDDEN_SIZE).cuda()

In [186]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=0.0025)

In [187]:
all(param.is_cuda for param in rnn.parameters())

True

In [188]:
epochs = 50
for epoch in range(epochs):
    for x, y in train_dl:
        outputs = rnn(x)
        loss_value = loss(outputs, y)
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

    print(f"Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}")

Эпоха 1, Значение функции потерь: 1.4311494827270508
Эпоха 2, Значение функции потерь: 1.383643388748169
Эпоха 3, Значение функции потерь: 1.267279863357544
Эпоха 4, Значение функции потерь: 1.2832894325256348
Эпоха 5, Значение функции потерь: 1.2918986082077026
Эпоха 6, Значение функции потерь: 1.6374574899673462
Эпоха 7, Значение функции потерь: 1.4478998184204102
Эпоха 8, Значение функции потерь: 1.5597985982894897
Эпоха 9, Значение функции потерь: 1.320909857749939
Эпоха 10, Значение функции потерь: 1.2415077686309814
Эпоха 11, Значение функции потерь: 1.457364797592163
Эпоха 12, Значение функции потерь: 1.5204041004180908
Эпоха 13, Значение функции потерь: 1.3025500774383545
Эпоха 14, Значение функции потерь: 1.0718073844909668
Эпоха 15, Значение функции потерь: 0.9505273103713989
Эпоха 16, Значение функции потерь: 1.4744417667388916
Эпоха 17, Значение функции потерь: 0.7291814088821411
Эпоха 18, Значение функции потерь: 1.047696828842163
Эпоха 19, Значение функции потерь: 1.12313

In [203]:
sm = nn.Softmax(1).cuda()

In [208]:
y_pred = torch.argmax(sm(rnn(X_test.cuda())).cpu(), dim=1)

print(classification_report(
    y_test,
    y_pred.cpu().detach().numpy(),
))

              precision    recall  f1-score   support

           0       0.58      1.00      0.74       584
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00        13
           3       0.00      0.00      0.00        13
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00        37
           6       0.00      0.00      0.00        31
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         7
          11       0.00      0.00      0.00       236
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00        26
          14       0.00      0.00      0.00         5

    accuracy                           0.58      1000
   macro avg       0.04      0.07      0.05      1000
weighted avg       0.34   

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined 

#### LSTM

In [221]:
class LSTMModel(nn.Module):
    def __init__(self, input_size: int, hidden_size: int) -> None:
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 15)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, (h, c) = self.lstm(x)
        y = self.linear(h.squeeze(0))
        return y

lstm = LSTMModel(INPUT_SIZE, HIDDEN_SIZE).cuda()

In [222]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.0025)

In [223]:
epochs = 50
for epoch in range(epochs):
    for x, y in train_dl:
        outputs = lstm(x)
        loss_value = loss(outputs, y)
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

    print(f"Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}")

Эпоха 1, Значение функции потерь: 1.0545998811721802
Эпоха 2, Значение функции потерь: 1.4772090911865234
Эпоха 3, Значение функции потерь: 1.146073341369629
Эпоха 4, Значение функции потерь: 1.432945728302002
Эпоха 5, Значение функции потерь: 1.4908427000045776
Эпоха 6, Значение функции потерь: 1.0412677526474
Эпоха 7, Значение функции потерь: 1.1031217575073242
Эпоха 8, Значение функции потерь: 1.521390676498413
Эпоха 9, Значение функции потерь: 1.4805948734283447
Эпоха 10, Значение функции потерь: 1.220950722694397
Эпоха 11, Значение функции потерь: 1.25678288936615
Эпоха 12, Значение функции потерь: 1.269113302230835
Эпоха 13, Значение функции потерь: 1.6073312759399414
Эпоха 14, Значение функции потерь: 1.3458689451217651
Эпоха 15, Значение функции потерь: 1.6384025812149048
Эпоха 16, Значение функции потерь: 1.4180195331573486
Эпоха 17, Значение функции потерь: 1.243863821029663
Эпоха 18, Значение функции потерь: 1.823279619216919
Эпоха 19, Значение функции потерь: 1.589805364608

In [224]:
sm = nn.Softmax(1).cuda()

In [226]:
y_pred = torch.argmax(sm(lstm(X_test.cuda())).cpu(), dim=1)

print(classification_report(
    y_test,
    y_pred.cpu().detach().numpy(),
))

              precision    recall  f1-score   support

           0       0.58      1.00      0.74       584
           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00        13
           3       0.00      0.00      0.00        13
           4       0.00      0.00      0.00         4
           5       0.00      0.00      0.00        37
           6       0.00      0.00      0.00        31
           7       0.00      0.00      0.00         4
           8       0.00      0.00      0.00        20
           9       0.00      0.00      0.00         4
          10       0.00      0.00      0.00         7
          11       0.00      0.00      0.00       236
          12       0.00      0.00      0.00        11
          13       0.00      0.00      0.00        26
          14       0.00      0.00      0.00         5

    accuracy                           0.58      1000
   macro avg       0.04      0.07      0.05      1000
weighted avg       0.34   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### GRU

In [232]:
class GRUModel(nn.Module):
    def __init__(self, input_size: int, hidden_size: int) -> None:
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 15)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, h = self.gru(x)
        y = self.linear(h.squeeze(0))
        return y

gru = GRUModel(INPUT_SIZE, HIDDEN_SIZE).cuda()

In [233]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.0025)

In [234]:
epochs = 50
for epoch in range(epochs):
    for x, y in train_dl:
        outputs = gru(x)
        loss_value = loss(outputs, y)
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

    print(f"Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}")

Эпоха 1, Значение функции потерь: 1.490487813949585
Эпоха 2, Значение функции потерь: 1.6580743789672852
Эпоха 3, Значение функции потерь: 1.1968626976013184
Эпоха 4, Значение функции потерь: 1.2106678485870361
Эпоха 5, Значение функции потерь: 0.6336907148361206
Эпоха 6, Значение функции потерь: 0.24605122208595276
Эпоха 7, Значение функции потерь: 0.3169252574443817
Эпоха 8, Значение функции потерь: 0.18240150809288025
Эпоха 9, Значение функции потерь: 0.2973821759223938
Эпоха 10, Значение функции потерь: 0.2070135623216629
Эпоха 11, Значение функции потерь: 0.25355297327041626
Эпоха 12, Значение функции потерь: 0.4475637376308441
Эпоха 13, Значение функции потерь: 0.12715286016464233
Эпоха 14, Значение функции потерь: 0.19285225868225098
Эпоха 15, Значение функции потерь: 0.0647289901971817
Эпоха 16, Значение функции потерь: 0.16539506614208221
Эпоха 17, Значение функции потерь: 0.4474617838859558
Эпоха 18, Значение функции потерь: 0.04484887793660164
Эпоха 19, Значение функции поте

In [235]:
sm = nn.Softmax(1).cuda()

In [236]:
y_pred = torch.argmax(sm(gru(X_test.cuda())).cpu(), dim=1)

print(classification_report(
    y_test,
    y_pred.cpu().detach().numpy(),
))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94       584
           1       0.57      0.80      0.67         5
           2       0.90      0.69      0.78        13
           3       0.71      0.77      0.74        13
           4       0.00      0.00      0.00         4
           5       0.88      0.95      0.91        37
           6       0.86      0.77      0.81        31
           7       1.00      0.50      0.67         4
           8       0.83      0.75      0.79        20
           9       1.00      0.25      0.40         4
          10       0.75      0.86      0.80         7
          11       0.88      0.90      0.89       236
          12       0.71      0.45      0.56        11
          13       0.77      0.65      0.71        26
          14       0.80      0.80      0.80         5

    accuracy                           0.90      1000
   macro avg       0.77      0.67      0.70      1000
weighted avg       0.90   